In [ ]:
!pip install -q git+https://github.com/tensorflow/docs

In [ ]:
!wget -q https://git.io/JGc31 -O ucf101_top5.tar.gz
!tar xf ucf101_top5.tar.gz

In [ ]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

## Hyper parameters

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 10

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

## Data preparation

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

print(f'Total videos for training: {len(train_df)}')
print(f'Total videos for test: {len(test_df)}')

train_df.sample(5)

## Build the CNN feature extractor

In [ ]:
def build_feature_extractor():
  feature_extractor = keras.applications.InceptionV3(
    weights='imagenet',
    include_top=False,
    pooling='avg',
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
  )
  preprocess_input = keras.applications.inception_v3.preprocess_input

  inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
  preprocessed = preprocess_input(inputs)

  outputs = feature_extractor(preprocessed)
  return keras.Model(inputs, outputs, name='feature_extractor')

In [ ]:
feature_extractor = build_feature_extractor()

In [ ]:
# convert label of the videos to numerical values
label_processor = keras.layers.StringLookup(
  num_oov_indices=0, vocabulary=np.unique(train_df['tag'])
)
print(label_processor.get_vocabulary())